In [12]:
import re
from difflib import get_close_matches

# Phonetic encoding rules for Tamil
def phonetic_encode(word):
    replacements = {
        'அ': 'a', 'ஆ': 'aa', 'இ': 'i', 'ஈ': 'ii', 'உ': 'u', 'ஊ': 'uu',
        'எ': 'e', 'ஏ': 'ee', 'ஐ': 'ai', 'ஒ': 'o', 'ஓ': 'oo', 'ஔ': 'au',
        'க': 'k', 'ச': 's', 'ட': 't', 'த': 'th', 'ப': 'p', 'ற': 'r',
        'ன': 'n', 'ந': 'n', 'ம': 'm', 'ய': 'y', 'ர': 'r', 'ல': 'l',
        'வ': 'v', 'ழ': 'zh', 'ள': 'l', 'ஜ': 'j', 'ஷ': 'sh', 'ஸ': 's', 'ஹ': 'h'
    }

    encoded_word = ""
    for char in word:
        encoded_word += replacements.get(char, char)

    return encoded_word

# Spell checker function
def spell_checker(input_word, tamil_dictionary):
    input_encoded = phonetic_encode(input_word)
    dictionary_encoded = {word: phonetic_encode(word) for word in tamil_dictionary}

    # Find closest matches based on phonetic encoding
    closest_matches = get_close_matches(input_encoded, dictionary_encoded.values(), n=3, cutoff=0.6)

    # Get original Tamil words for the matches
    suggestions = [
        word for word, encoded in dictionary_encoded.items() if encoded in closest_matches
    ]

    return suggestions

# Grammar rules
def check_grammar(sentence):
    GRAMMAR_RULES = {
        "simple_sentence": re.compile(r"^(?:\u0BA8\u0BBE\u0BA9\u0BCD|\u0BA8\u0BC0|\u0BAA\u0B9F\u0BAE\u0BCD|\u0BAE\u0BB4\u0BC8|\u0BAE\u0BB2\u0BB0\u0BCD|\u0BAF\u0BBE\u0BB0\u0BCD|\u0BB0\u0BAE\u0BA9\u0BCD)\s+\S+\s+(?:\u0B9A\u0BC8\u0BAF\u0BCD\u0B95\u0BBF\u0BB1\u0BC7\u0BA9\u0BCD|\u0B9A\u0BC8\u0BAF\u0BCD\u0B95\u0BBF\u0BB1\u0BCB\u0BA9\u0BCD)\.$")
    }

    issues = []

    # Check for missing punctuation
    if not sentence.endswith(".") and not sentence.endswith("\u0964"):
        issues.append("Sentence should end with a period (.) or a Tamil punctuation (।).")

    # Match against grammar rules
    for rule_name, rule in GRAMMAR_RULES.items():
        if rule.match(sentence):
            return "Sentence is grammatically correct."

    # If no grammar rule matches, flag an error
    issues.append("Sentence does not match any known grammar rule.")
    return issues

# Integrated spell and grammar checker
def integrated_checker(sentence, tamil_dictionary):
    words = sentence.split()
    all_suggestions = {}
    corrected_sentence = []

    # Check each word for spelling
    for word in words:
        suggestions = spell_checker(word, tamil_dictionary)
        if suggestions and word not in tamil_dictionary:
            all_suggestions[word] = suggestions
            corrected_sentence.append(suggestions[0])  # Use the first suggestion as correction
        else:
            corrected_sentence.append(word)

    # Reconstruct corrected sentence
    corrected_sentence = " ".join(corrected_sentence)

    # Ensure punctuation at the end
    if not (corrected_sentence.endswith(".") or corrected_sentence.endswith("\u0964")):
        corrected_sentence += "."

    # Check grammar
    grammar_result = check_grammar(corrected_sentence)

    return all_suggestions, grammar_result, corrected_sentence

# Load Tamil dictionary from external file
def load_tamil_dictionary(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        tamil_dictionary = [line.strip() for line in file]
    return tamil_dictionary

# Example usage
if __name__ == "__main__":
    # Load dictionary from file
    tamil_dictionary = load_tamil_dictionary("C:/Users/Admin/Desktop/dictionary.txt")

    # List of sentences to check
    sentences = [
        "நான் படம் படிக்கிறே",
        "அனைவருக்கும் நல்வாழ்துகள். தமிழின் மிக்க பழமயான மொழி வரலாற்றையும் பெருமையும் கொண்டது. இது உலகன் மிக அழகிய மற்றும் செம்மொழிகளில ஒன்றாகும். தமிழில் சொற்களை சரயாக எழுதுவதும், சரியான இடத்தில முறையாக பயன்படுத்துதும் மிகவும் முக்கியமானவை"
    ]

    # Process and output results for each sentence
    for sentence in sentences:
        print(f"Checking sentence: {sentence}")
        spell_suggestions, grammar_check, corrected_sentence = integrated_checker(sentence, tamil_dictionary)

        # Output results
        print("\nSpell Check Suggestions:")
        for word, suggestions in spell_suggestions.items():
            print(f"  {word}: {', '.join(suggestions)}")

        print("\nGrammar Check Result:")
        if isinstance(grammar_check, str):
            print(grammar_check)
        else:
            print("\n".join(grammar_check))

        print("\nCorrected Sentence:")
        print(corrected_sentence)
        print("\n" + "-"*50 + "\n")


Checking sentence: நான் படம் படிக்கிறே

Spell Check Suggestions:
  படம்: பாடம், பணம், இடம்
  படிக்கிறே: படிக்கிறேன், படிக்க

Grammar Check Result:
Sentence does not match any known grammar rule.

Corrected Sentence:
நான் பாடம் படிக்கிறேன்.

--------------------------------------------------

Checking sentence: அனைவருக்கும் நல்வாழ்துகள்! தமிழின் மிக்க பழமயான மொழி வரலாற்றையும் பெருமையும் கொண்டது. இது உலகன் மிக அழகிய மற்றும் செம்மொழிகளில ஒன்றாகும். தமிழில் சொற்களை சரயாக எழுதுவதும், சரியான இடத்தில முறையாக பயன்படுத்துதும் மிகவும் முக்கியமானவை

Spell Check Suggestions:
  நல்வாழ்துகள்!: நல்வாழ்த்துகள்
  பழமயான: பழமையான
  கொண்டது.: குறைந்தது, கொண்டது
  உலகன்: உலகம், உகந்த, உலகின்
  செம்மொழிகளில: செய்திகள், சமயங்களில், செம்மொழிகளில்
  ஒன்றாகும்.: நாகம், நாளும், ஒன்றாகும்
  சரயாக: சரியாக, சரியான, முறையாக
  எழுதுவதும்,: எழுத்து, சுத்தம், எழுதுவதும்
  இடத்தில: ஒத்திகை, தொட்டில், இடத்தில்
  பயன்படுத்துதும்: புத்தகம், சேதப்படுத்தல், பயன்படுத்துவதும்

Grammar Check Result:
Sentence does not match any